# PLNCD - Curso 2023-24

## Tarea 3: Intent Classification

### Enunciado de la tarea

En esta tarea se debe implementar una serie de algoritmos para llevar a cabo una tarea de clasificación de intents.

La tarea de intent classification consiste en identificar la intención o el propósito asociado a un texto o una consulta dada. Por lo general, se aplica en sistemas de procesamiento de lenguaje natural (NLP) y en aplicaciones de procesamiento de texto, como asistentes virtuales, sistemas de atención al cliente y motores de búsqueda, entre otros.

El objetivo principal de la clasificación de intenciones es entender lo que un usuario intenta hacer o expresar a partir de su entrada de texto. Por ejemplo, en un sistema de asistente virtual, si un usuario escribe "¿Va a llover hoy?", la tarea de clasificación de intenciones podría identificar la intención del usuario como "Consultar tiempo". Esta información puede utilizarse para dirigir al usuario al servicio adecuado o proporcionar la respuesta correcta.

Para realizar la tarea, vamos a utilizar un dataset que ya ha sido preparado, y que contiene 150 intents.

El fichero de entrenamiento asociado a esta tarea se denomina:

"Dataset-Intent-Train.csv"

y contiene un listado de frases, cada una de las cuales está asociada con su intención correspondiente.

Por ejemplo:

```
new_card,"is there a list of good credit cards i can choose from to apply for"
new_card,"how can i find a new credit card to apply to"
lost_luggage,"who do i contact for lost luggage"
lost_luggage,"where is my luggage"
```

Como se puede observar, cada entrada comienza con el descriptor del intent, y a continuación, separado por una coma, aparece el texto de la frase correspondiente.

Además de las intenciones básicas (correspondientes a operaciones concretas como pedir una nueva tarjeta o preguntar por el equipaje perdido en un viaje), el problema de la clasificación de intenciones tiene el reto de las frases fuera del dominio. Es decir, si estamos trabajando en el dominio bancario, podemos tener intenciones tales como consultar el saldo, hacer una transferencia, etc., pero si alguien se confunde y usa el sistema para consultar el resultado de un partido de fútbol, muy probablemente el sistema se confunda y devuelva una intención del dominio bancario, pero que será incorrecto.

Así que es importante tener en cuenta cómo responden los sistemas ante frases fuera de dominio (Out Of Scope - oos). Por eso en este dataset se incluyen un conjunto extenso de frases oos.

Esta tarea consistirá en crear una clase, siguiendo el patrón que se muestra de ejemplo a continuación.

La clase permitirá en entrenamiento de un sistema de clasificación de intenciones, y la evaluación de un conjunto de tests.

La clase permitirá varios algoritmos para trabajar. Como ejemplo se muestra un algoritmo totalmente inválido, que simplemente genera una intent al azar.

Pero se debe seguir el patrón que se indica para resolver el ejercicio.

En concreto, la clase debe permitir los siguientes métodos:

* __Constructor__: Simplemente crea un objeto de la clase IntentClassification 
* __getIdentification__: Devuelve los datos de identificación de la persona que ha realizado el ejercicio
* __loadDataset__: carga un dataset. Se puede utilizar la implementación adjunta ya que lo que se realiza es instanciar todos los intents detectados en el dataset y asocia con cada intent sus frases de ejemplo asociadas
* __getIntents__: devuelve la lista de todas las intents cargadas en un momento determinado
* __trainIntentClassification__: Se deben implementar los algoritmos de entrenamiento correspondientes
* __intentClassification__: Se deben implementar los algoritmos correspondientes de clasificación
* __testClassification__: Recorre un fichero de evaluación y aplicando el método de clasificación (__intentClassification__) va construyendo el resultado. La implementación facilitada debería ser válida con carácter general.
* __testSummary__: Devuelve los datos básicos de una evaluación: total de ejemplos evaluados, total de correctos y porcentaje general. Debería ser suficiente la implementación facilitada.
* __printTestResults__: Imprime un informe exhaustivo indicando el total de ejemplos evaluados, correctos y porcentaje, para cada intent. Debería ser suficiente la implementación facilitada


Como parte de la implementación ejemplo, se ha implementado un algoritmo RANDOM, que realmente no hace entrenamiento, y que para hacer la clasificación lo que hace es devolver una intención al azar de entre las disponibles.

La primera fase de este trabajo consiste en implementar un algoritmo que aplique un modelo de clasificación utilizando la estrategia Naive Bayes (código __IntentClassification.BAYES__), y en segundo lugar un algoritmo basado en TfIdf (código __IntentClassification.TFIDF__)

Se debe entregar este mismo fichero, renombrado de la siguiente forma:

PLNCD-Apellidos-Nombre-Tarea3.ipynb



In [1]:
# Preview de los datos
# def loadDataset(file):
#     # Temporalmente, voy a hacer que solo carge los 5 primeros
#     f = open(file, "r", encoding='utf-8')
#     dataset = {}

#     count = 0
    
#     for line in f.readlines():
#         sep = line.find(',')
#         intent = line[:sep]
#         sentence = line[sep+1:]
#         samples = dataset.get(intent,[])
#         samples.append(sentence)
#         dataset[intent] = samples

#         # --- TEMPORAL ---
#         count += 1
#         if count == 5:
#             break
#     intents = list(dataset.keys())

file = 'Dataset-Intent-Train.csv'
f = open(file, "r", encoding='utf-8')
dataset = {}

# count = 0

for line in f.readlines():
    # print(line)
    sep = line.find(',')
    intent = line[:sep]
    sentence = line[sep+1:]
    samples = dataset.get(intent,[])
    samples.append(sentence)
    dataset[intent] = samples

    # print(dataset)

    # # --- TEMPORAL ---
    # count += 1
    # if count == 5:
    #     break

intents = list(dataset.keys())
print(dataset)
print(intents)

{'oos': ['"a show on broadway"\n', '"who has the best record in the nfl"\n', '"how do i find the area of a circle"\n', '"what is the name of the 13th president"\n', '"does our bank have free notary"\n', '"what were the top stories this week"\n', '"when was the last time the president visited floridau"\n', '"how do i compute the median of a set of numbers"\n', '"can you tell me how to compute price per ounce"\n', '"how to unclog a drain"\n', '"what new reality shows are there for 2019"\n', '"what is the purpose or benefit of a cash advance"\n', '"how can i root an android phone"\n', '"is android better than iphone"\n', '"are androids expensive"\n', '"who are some famous living architects"\n', '"find out for me as to why people get autism"\n', '"is autism a genetic disease"\n', '"is flu contagious"\n', '"king of acid"\n', '"suo2"\n', '"what university in the united states offers the best biomedical engineering program"\n', '"when was electrical engineering created"\n', '"is it possible t

#### Datos Identificativos:

##### Apellidos y Nombre: Parrales de la Cruz, Domingo

In [2]:
import random

# --- Tokenización / preprocesado ---
import nltk
from nltk import word_tokenize # separar signos de puntuación (inglés)
# Naive Bayes
from nltk import NaiveBayesClassifier

# TF-IDF
from nltk.stem.lancaster import LancasterStemmer # Estemización
from nltk.corpus import stopwords # stopwords: what, in, the, on...
import string # punctuation
import math # log, cosine
from operator import itemgetter

In [3]:
class tfidf():
    def __init__(self):
        self.vocabulario = []
        self.categorias = []
        self.idfs = {}
        self.vectores_categorias = {}

    # --- Atributos para hacer cálculos dentro de la clase ---
    def modulo(self, vector):
        modulo_al_cuadrado = 0
        for coordenada in vector:
            modulo_al_cuadrado += math.pow(coordenada, 2)
        return math.sqrt(modulo_al_cuadrado)
        
    def dot_product(self, vector1, vector2):
        if len(vector1) == len(vector2):
            dot_prod = 0
            for index in range(0, len(vector1)):
                dot_prod += vector1[index]*vector2[index]
            return dot_prod
        else:
            return "Unmatching dimensionality"
    
    def calculate_cosine(self, test_document, category):
        modulo_test = self.modulo(test_document)
        if modulo_test == 0:
            return 0
        
        return self.dot_product(test_document, category) / (modulo_test * self.modulo(category)) 

    def lista_categorias_ordenada(self, test_document, categorias):
        results = {}
    
        for categoria in categorias.keys():
            cosine = self.calculate_cosine(test_document, categorias.get(categoria))    
            results[categoria] = cosine
    
        search = []
        for items in sorted(results.items(), key=itemgetter(1), reverse=True):
            search.append(items)
        return search[0]
    
    # --- Entrenamiento del modelo ---
    def train(self, rasgos):
        # 1. Vocabulario completo de train 
        self.vocabulario = []
        n_documentos = 0 # aprovecho para contar el número de documentos
        self.categorias = [] # también extraigo la lista de categorías del conjunto train
        
        for (documento, categoria ) in rasgos:
            n_documentos += 1 
            self.categorias.append(categoria)
            for termino in documento.keys():
                self.vocabulario.append(termino)

        self.vocabulario = sorted(set(self.vocabulario))
        self.categorias = set(self.categorias)

        # Calcular IDFs 
        self.idfs = {}

        for termino in self.vocabulario:
            contador_apariciones = 0
            
            for idx in range(len(rasgos)):
                if termino in rasgos[idx][0].keys():
                    contador_apariciones += 1

            self.idfs[termino] = math.log( 
                int(n_documentos) / int(1 + contador_apariciones),
                10
            )

        # Vectorizar categorías 
        self.vectores_categorias = {}

        for categoria in self.categorias:
            # Inicializar diccionario
            self.vectores_categorias[categoria] = [0 for _ in range(len(self.vocabulario))]
            
        for (documento, categoria) in rasgos:
            output_vector = []
            
            # Coordenadas de cada documento
            for word in self.vocabulario:
                if word in documento.keys():
                    output_vector.append(int(documento[word]))
                else:
                    output_vector.append(0)

            # Sumamos la aportación de cada documento
            self.vectores_categorias[categoria] = [x+y for (x,y) in zip(self.vectores_categorias[categoria], output_vector)]

    # --- Clasificación de test ---
    def classify(self, featureset):
        # Vectorizar tf_idf de test
        sample_vectorizado = []
        
        for term in self.vocabulario:
            doc_count = 0

            # Vector TF
            if term in featureset.keys():
                sample_vectorizado.append(
                    featureset[term] * self.idfs[term]
                    )
            else:
                sample_vectorizado.append(0)

        # Clasificación
        return self.lista_categorias_ordenada(sample_vectorizado,
                                         self.vectores_categorias)[0]

In [4]:
class IntentClassification():
    RANDOM = 0
    BAYES = 1
    TFIDF = 2
    
    def __init__(self):
        self.method = IntentClassification.RANDOM
        self.dataset = {}
        self.intents = []
        self.test = {}
        self.classifier = NaiveBayesClassifier
        
    def getIdentification(self):
        return ("Parrales de la Cruz, Domingo", "47549947T")
        
    def loadDataset(self, file):
        # f = open(file, "r", encoding='utf-8')
        # Prefiero usar with(open(...)), porque hay que cerrar el documento 
        
        self.dataset = {}

        with open(file, 'r', encoding='utf-8') as f:
            for line in f.readlines():
                sep = line.find(',')
                intent = line[:sep]
                sentence = line[sep+1:]
                samples = self.dataset.get(intent,[])
                samples.append(sentence)
                self.dataset[intent] = samples

        self.intents = list(self.dataset.keys())
        
    def getIntents(self):
        return self.intents

    # Preprocesado / tokenización
    def obtener_rasgos(self, text):
        # --- Preprocesado para TF-IDF ---
        if self.method == IntentClassification.TFIDF:
            # Stopwords a eliminar
            stoplist = set(stopwords.words('english'))
            
            # Estemización : consider -> consid
            st = LancasterStemmer()
            
            # Preprocesado 
            word_list = [st.stem(word) for word in word_tokenize(text.lower())
                          if not word in stoplist and 
                          not word in string.punctuation]
            
            # Contamos el número de apariciones
            terms = {}
            for word in word_list:
                terms[word] = terms.get(word, 0) + 1 
            return terms # pruebas para ver progreso

        # Preprocesado para Naive Bayes 
        # Crea un diccionario en el que asigna a las palabras que aparecen en el 
        # texto (text) el valor True
        rasgos = {}
        tokens = [palabra for palabra in word_tokenize(text.lower())] # lo pasa a minúsulas
        for token in tokens:
            # Para cada palabra hace un diccionario en la que pone True si aparece la palabra
            rasgos[token] = True
        return rasgos

    # Entrenamiento de los modelos
    def trainIntentClassification(self):
        rasgos = [
            (self.obtener_rasgos(texto), categoria)
            for categoria in self.intents
            for texto in self.dataset[categoria]
        ]
        
        # Implementar el algoritmo de clasificación y realizar el entrenamiento
        if self.method == IntentClassification.BAYES:
            # Construimos modelo naive bayes con datos de entrenamiento 
            self.classifier = NaiveBayesClassifier.train(rasgos)

        
        elif self.method == IntentClassification.TFIDF:
            # Entrenamiento del modelo tf-idf
            self.classifier = tfidf()
            self.classifier.train(rasgos)

    # Clasificación de un texto a una categoría
    def intentClassification(self, sample):
        if (len(self.intents) == 0):
            return 'None'
            
        # --- Random ---
        if self.method == IntentClassification.RANDOM:
            r = random.randint(0,len(self.intents)-1)
            return self.intents[r]
                
        # --- Naive Bayes y TF-IDF---  
        elif self.method == IntentClassification.BAYES or IntentClassification.TFIDF:            
            # Feature extraction: extraer propiedades o características relevantes
            sample_preprocesado = self.obtener_rasgos(sample)

            # Predicción de test 
            return self.classifier.classify(featureset = sample_preprocesado)

        else:
            raise ValueError('Método no reconocido \nLos métodos válidos son: 0 (RANDOM), 1 (NAIVE BAYES), 2 (TF-IDF)')

    # Clasificación del conjunto test
    def testClassification(self, file, method):
        self.method = method
        self.trainIntentClassification() # entrenamos modelo con train
        
        self.test = {}

        # --- Preparamos almacenamiento para cada categoría ---
        for i in self.dataset: # igual que poner i in self.dataset.keys()
            self.test[i] = {"samples": 0, "correct": 0}

        # --- Documentos de test ---
        # f = open(file, "r", encoding='utf-8')
        # Prefiero usar with(open(...)), porque hay que cerrar el documento 

        # Contar número de líneas (medir progreso)
        with(open(file, "rb") as f):
            num_lines = sum(1 for _ in f)

        # Recorrer líneas para predecir categorías
        with(open(file, 'r', encoding='utf-8') as f):
            count = 0 # medir progreso
            for line in f.readlines():
                # --- Cargar línea ---
                sep = line.find(',')
                goldIntent = line[:sep]
                sentence = line[sep+1:]

                # --- Clasificación y almacenamiento del resultado ---
                classifiedIntent = self.intentClassification(sentence)
                
                self.test[goldIntent]["samples"] += 1
                if goldIntent == classifiedIntent:
                    self.test[goldIntent]["correct"] += 1

                # Indicar progreso
                count += 1
                print('\r', f'{count} / {num_lines}', end='')

    
    # Mostrar resultados (rendimiento del modelo)
    def testSummary(self):
        # --- Cálculo del rendimiento global ---
        totalSamples = 0
        totalCorrect = 0
        for i in self.test:
            totalSamples += self.test[i]["samples"]
            totalCorrect += self.test[i]["correct"]

        return (totalSamples, totalCorrect, 
                100 * totalCorrect / totalSamples)
    
    def printTestResults(self):
        maxIntentDescriptor = 0
        for i in self.dataset:
            if len(i) > maxIntentDescriptor:
                maxIntentDescriptor = len(i)

        print('\n')
        print(f"   | {'Intent':{maxIntentDescriptor}} | Samples | Correct | Perc.   | ")
        print( "   |" + ('-' * (maxIntentDescriptor)) + '--|---------|---------|---------|' )
        totalSamples = 0
        totalCorrect = 0
        for i in self.test:
            samples = self.test[i]["samples"]
            totalSamples += samples
            correct = self.test[i]["correct"]
            totalCorrect += correct
            perc    = round(correct / samples, 2)
            
            print(f"   | {i:{maxIntentDescriptor}} | {samples:7} | {correct:7} | {perc:7} |")
            
        print( "   |" + ('-' * (maxIntentDescriptor)) + '--|---------|---------|---------|' )
        totalPerc = round(totalCorrect/totalSamples, 2)
        print(f"   | {'Total':{maxIntentDescriptor}} | {totalSamples:7} | {totalCorrect:7} | {totalPerc:7} | ")
        print( "   |" + ('-' * (maxIntentDescriptor)) + '--|---------|---------|---------|' )
        
        

Evaluación TF-IDF:

In [5]:
ic = IntentClassification()
ic.loadDataset('Dataset-Intent-Train.csv')

# Evaluación TF-IDF
ic.testClassification("Dataset-Intent-Test.csv", IntentClassification.TFIDF)
ic.testSummary()
ic.printTestResults()

 2351 / 2351

   | Intent                    | Samples | Correct | Perc.   | 
   |---------------------------|---------|---------|---------|
   | oos                       |     187 |       3 |    0.02 |
   | translate                 |      16 |      13 |    0.81 |
   | transfer                  |      15 |      13 |    0.87 |
   | timer                     |      18 |      15 |    0.83 |
   | definition                |      16 |      12 |    0.75 |
   | meaning_of_life           |      18 |      16 |    0.89 |
   | insurance_change          |      13 |      13 |     1.0 |
   | find_phone                |      17 |      11 |    0.65 |
   | travel_alert              |      15 |      14 |    0.93 |
   | pto_request               |       9 |       5 |    0.56 |
   | improve_credit_score      |      16 |      15 |    0.94 |
   | fun_fact                  |      13 |      11 |    0.85 |
   | change_language           |      14 |       7 |     0.5 |
   | payday                    |      17

Evaluación Naive Bayes:

In [6]:
ic = IntentClassification()
ic.loadDataset('Dataset-Intent-Train.csv')

# Evaluación Naive Bayes
ic.testClassification("Dataset-Intent-Test.csv", IntentClassification.BAYES)
ic.testSummary()
ic.printTestResults()

 2351 / 2351

   | Intent                    | Samples | Correct | Perc.   | 
   |---------------------------|---------|---------|---------|
   | oos                       |     187 |       5 |    0.03 |
   | translate                 |      16 |      15 |    0.94 |
   | transfer                  |      15 |      11 |    0.73 |
   | timer                     |      18 |      16 |    0.89 |
   | definition                |      16 |      12 |    0.75 |
   | meaning_of_life           |      18 |      17 |    0.94 |
   | insurance_change          |      13 |      13 |     1.0 |
   | find_phone                |      17 |      13 |    0.76 |
   | travel_alert              |      15 |      12 |     0.8 |
   | pto_request               |       9 |       8 |    0.89 |
   | improve_credit_score      |      16 |      15 |    0.94 |
   | fun_fact                  |      13 |      10 |    0.77 |
   | change_language           |      14 |      10 |    0.71 |
   | payday                    |      17

Otras pruebas:

In [7]:
ic = IntentClassification()

In [8]:
ic.getIdentification()

('Parrales de la Cruz, Domingo', '47549947T')

In [9]:
ic.loadDataset('Dataset-Intent-Train.csv')

In [10]:
ic.getIntents()[:10]

['oos',
 'translate',
 'transfer',
 'timer',
 'definition',
 'meaning_of_life',
 'insurance_change',
 'find_phone',
 'travel_alert',
 'pto_request']

In [11]:
ic.intentClassification("hello good morning")

'insurance'

In [12]:
# En el caso anterior, se clasifica usando RANDOM. Para usar otro método, 
# como el Naive Bayes, tenemos que entrenar previamente el modelo:

ic.method = IntentClassification.BAYES
ic.trainIntentClassification()
ic.intentClassification('hello good morning')

'translate'

In [13]:
ic.intentClassification('My account is not working')

'account_blocked'

In [14]:
ic.intentClassification('alarm clock')

'alarm'

In [15]:
ic.testClassification("Dataset-Intent-Test.csv",IntentClassification.RANDOM)

 2351 / 2351

In [16]:
ic.testSummary()

(2351, 10, 0.42535091450446616)

In [17]:
ic.printTestResults()



   | Intent                    | Samples | Correct | Perc.   | 
   |---------------------------|---------|---------|---------|
   | oos                       |     187 |       0 |     0.0 |
   | translate                 |      16 |       0 |     0.0 |
   | transfer                  |      15 |       0 |     0.0 |
   | timer                     |      18 |       0 |     0.0 |
   | definition                |      16 |       0 |     0.0 |
   | meaning_of_life           |      18 |       0 |     0.0 |
   | insurance_change          |      13 |       0 |     0.0 |
   | find_phone                |      17 |       0 |     0.0 |
   | travel_alert              |      15 |       0 |     0.0 |
   | pto_request               |       9 |       0 |     0.0 |
   | improve_credit_score      |      16 |       0 |     0.0 |
   | fun_fact                  |      13 |       0 |     0.0 |
   | change_language           |      14 |       0 |     0.0 |
   | payday                    |      17 |       0 |